In [3]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import math
import seaborn as sns
import sklearn
from sklearn import linear_model
from sklearn import preprocessing
%matplotlib inline
sns.set_style('white')

Use this dataset of airline arrival information to predict how late flights will be. A flight only counts as late if it is more than 30 minutes late.

In [4]:
df = pd.read_csv('2008.csv')

In [3]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,...,4.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,...,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,...,3.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,...,3.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,...,3.0,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0


7	ArrTime	actual arrival time (local, hhmm)

8	CRSArrTime	scheduled arrival time (local, hhmm)

In [4]:
df['late'] = np.where(df['ArrTime'] - df['CRSArrTime'] > 30, 1, 0)

In [5]:
df['late'].value_counts()

0    5812213
1    1197515
Name: late, dtype: int64

# 1 = Late, 0 = On Time

In [6]:
df.dtypes

Year                   int64
Month                  int64
DayofMonth             int64
DayOfWeek              int64
DepTime              float64
CRSDepTime             int64
ArrTime              float64
CRSArrTime             int64
UniqueCarrier         object
FlightNum              int64
TailNum               object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin                object
Dest                  object
Distance               int64
TaxiIn               float64
TaxiOut              float64
Cancelled              int64
CancellationCode      object
Diverted               int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
late                   int64
dtype: object

In [7]:
df_2 = df.drop(['UniqueCarrier', 'TailNum', 'Origin', 'Dest', 'CancellationCode'], axis=1)

In [8]:
df_2.dtypes

Year                   int64
Month                  int64
DayofMonth             int64
DayOfWeek              int64
DepTime              float64
CRSDepTime             int64
ArrTime              float64
CRSArrTime             int64
FlightNum              int64
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Distance               int64
TaxiIn               float64
TaxiOut              float64
Cancelled              int64
Diverted               int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
late                   int64
dtype: object

In [9]:
LATE = df_2['late']

In [10]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier(n_jobs=-1)
X = df_2.drop('late', 1)
Y = df_2['late']
X = X.dropna(axis=1)

In [11]:
features = X.columns
rfc.fit(X, Y)
importances = rfc.feature_importances_
indices = np.argsort(importances)

In [12]:
for feature in zip(features, rfc.feature_importances_): 
    print(feature)

('Year', 0.0)
('Month', 0.05867125781035039)
('DayofMonth', 0.18476175459468785)
('DayOfWeek', 0.07036098083763137)
('CRSDepTime', 0.14477302991004873)
('CRSArrTime', 0.1846982789124501)
('FlightNum', 0.18229362356494203)
('Distance', 0.16770612679708202)
('Cancelled', 0.0052442682539175075)
('Diverted', 0.001490679318890019)


In [11]:
df_3 = df_2[['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'CRSDepTime', 'CRSArrTime', 'FlightNum',
            'Distance', 'Cancelled']]

In [12]:
from sklearn import tree

# Packages for rendering our tree.
import pydotplus
import graphviz

# Initialize and train our tree.
decision_tree = tree.DecisionTreeClassifier(
    criterion='entropy',
    max_features=1,
    max_depth=4,
)
decision_tree.fit(df_3, LATE)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=1, max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [13]:
cross_dc = cross_val_score(decision_tree, df_3, LATE, cv=10)

In [14]:
cross_dc

array([0.82916342, 0.82916342, 0.82916342, 0.82916318, 0.82916318,
       0.82916436, 0.82916436, 0.82916436, 0.82916436, 0.82916436])

In [15]:
# took out 'Diverted', one score was bad before

In [16]:
rfc = ensemble.RandomForestClassifier(n_jobs= -1)
X = df_3
Y = LATE
X = X.dropna(axis=1)

In [17]:
cross_rf = cross_val_score(rfc, X, Y, cv=10)

In [18]:
cross_rf

array([0.74533863, 0.65019245, 0.62889208, 0.51982316, 0.31256696,
       0.1763594 , 0.37242714, 0.81568593, 0.82930274, 0.37278094])

Got 83% with the decision tree model and more scattered results with the random forest.  

In another notebook, I tried running pd.get_dummies for the columns I dropped in this notebook and got similar results.  pd.get_dummies(df['Tailnum']) and the feature importance formula took too long to be processed in a decent amount of time.  I adjusted accordingly, and that is probably why the results were similar to this notebook.